# Examples of OpenAI assistants

In [2]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import os

# en windows es necesario especificar la ruta del .env
load_dotenv(Path("C:\projects\rag\.env"))
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

***Power point example***

Assitant creation

In [3]:
assistant_name = "Power Point Generator"
output_file_name = "Presentation.pptx"
assistant_instruction = r"Generate {} file, always. You are subject-matter expert in the topic and professional in creating PowerPoints.. Betweem 1-5 slides. Background, colors, fonts and styling must be modern and easy to read. Make content engaging. Make the file id available to download.".format(output_file_name)
prompt_user = "Make a presentation for runner practicing for a half-marathon with an aim for a personal record. Make a presentaton with useful insights, training plan for different levels and some tips before, during and after the training period. Give insights."

assistant = client.beta.assistants.create(
    name=assistant_name,
    instructions=assistant_instruction,
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview")

Thread

In [4]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt_user)

Run, it runs assyncronous

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

Once the response is available, I can retrieve it.

In [6]:
import time

timeout = 180
interval_time = 5
time_taken = 0
while time_taken < timeout:
    run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id)

    if run.status == 'completed':
        break
    else:
        print(run.status)
        time.sleep(interval_time)
        time_taken += interval_time

# Alternatively solution is to run below simpler code and ensure run.status printed is 'completed', which can take up to 2 minutes:
#run = client.beta.threads.runs.retrieve(
#    thread_id=thread.id,
#    run_id=run.id)
#time.sleep(2)
# print(run.status)

Pull result

In [7]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_name = client.files.with_raw_response.retrieve_content(file_path)

with open(output_file_name, "wb") as file:  
    file.write(file_name.content) 

***Code interpreter***

In [8]:
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

In [9]:
thread = client.beta.threads.create()

In [10]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [11]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [12]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_jQx1ax5Om59J5Tr61typUfSB', assistant_id='asst_HjiGbwNwumjlb0aCTtVBpdIs', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Of course, Jane Doe! Let's solve the equation step-by-step:\n\nGiven the equation:\n\\[ 3x + 11 = 14 \\]\n\n1. Subtract 11 from both sides to isolate the term with \\( x \\):\n\\[ 3x + 11 - 11 = 14 - 11 \\]\n\\[ 3x = 3 \\]\n\n2. Divide both sides by 3 to solve for \\( x \\):\n\\[ \\frac{3x}{3} = \\frac{3}{3} \\]\n\\[ x = 1 \\]\n\nSo, the solution to the equation \\( 3x + 11 = 14 \\) is \\( x = 1 \\)."), type='text')], created_at=1725848049, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_xHvpSZA6Of2vfgbKclxIA0AG', status=None, thread_id='thread_wJafetM1TOycVjt8dn4LTh9f'), Message(id='msg_UcxtL8GipLDPalcGExHqLcXA', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotation

# Advance assistant

Example of a refactor assistant

In [17]:
file = client.files.create(
  file=open("sample.js", "rb"),
  purpose='assistants'
)

In [23]:
assistant = client.beta.assistants.create(
  name="Code writer assistant",
  description="Code writer assistant",  
  model="gpt-4o",
  instructions="""You are an expert software engineer. 
  Your goal is to help another engineer write code using Javascript. 
  Focus on writting clean, performant and concise code to solve the problem. 
  Your response should be in the followinf json format:
  {
    "description": "Your explanation here",
    "files":[
        {
            "filePath": "filepath/filename.extension",
            "fileContents": "file contents here"
        }
    ]
  }
  """,
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }  
)

In [24]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Refactor this file.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [25]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [26]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_NduhTjnsvKkaiWyRYfSk4h58', assistant_id='asst_VMZqGRm8uMlfSR0Qjg7oTA6e', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='{\n  "description": "Refactored Tic-Tac-Toe JavaScript game for better readability, performance, and maintainability.",\n  "files": [\n    {\n      "filePath": "tic-tac-toe/tic-tac-toe.js",\n      "fileContents": "const cells = document.querySelectorAll(\\".cell\\");\\nconst statusText = document.querySelector(\\"#statusText\\");\\nconst restartBtn = document.querySelector(\\"#restartBtn\\");\\n\\nconst winConditions = [\\n    [0, 1, 2],\\n    [3, 4, 5],\\n    [6, 7, 8],\\n    [0, 3, 6],\\n    [1, 4, 7],\\n    [2, 5, 8],\\n    [0, 4, 8],\\n    [2, 4, 6]\\n];\\n\\nlet options = Array(9).fill(\\"\\");\\nlet currentPlayer = \\"tomato\\";\\nlet running = false;\\n\\n// Initialize game\\ninitializeGame();\\n\\nfunction initializeGame() {\\n    cells.forEach((cell, index) => {\\n   

Parsing results. It works; however, sometimes the out cannot be parsed and we need another alternatives to downloiad it.

In [48]:
import json

output = messages.data[0].content[0].text.value
output_dict = json.loads(output)

print(output_dict['description'])
print(output_dict['files'][0]['filePath'])
print(output_dict['files'][0]['fileContents'])

with open("sample_refactored.js", "w") as file:  
    file.write(output_dict['files'][0]['fileContents']) 
file.close()


Refactored Tic-Tac-Toe JavaScript game for better readability, performance, and maintainability.
tic-tac-toe/tic-tac-toe.js
const cells = document.querySelectorAll(".cell");
const statusText = document.querySelector("#statusText");
const restartBtn = document.querySelector("#restartBtn");

const winConditions = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [0, 3, 6],
    [1, 4, 7],
    [2, 5, 8],
    [0, 4, 8],
    [2, 4, 6]
];

let options = Array(9).fill("");
let currentPlayer = "tomato";
let running = false;

// Initialize game
initializeGame();

function initializeGame() {
    cells.forEach((cell, index) => {
        cell.setAttribute("cellIndex", index);
    });
    document.querySelector(".gameContainer").addEventListener("click", cellClicked);
    restartBtn.addEventListener("click", restartGame);
    updateStatus();
    running = true;
}

function cellClicked(event) {
    const target = event.target;
    
    if (target.classList.contains("cell")) {
        const cellIndex

Create a response asking for unit test

In [49]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Write a simple unit test for that file")

In [50]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [51]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_nVizZmGtlralgqpLFmA1duhd', assistant_id='asst_VMZqGRm8uMlfSR0Qjg7oTA6e', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='To write a simple unit test for the Tic-Tac-Toe game, we\'ll use Jest, a popular JavaScript testing framework. We\'ll test the core functions such as `updateCell`, `changePlayer`, `checkWinner`, and `restartGame`.\n\nEnsure that you have Jest installed in your project:\n```bash\nnpm install --save-dev jest\n```\n\nHere\'s how you can create a unit test file for the Tic-Tac-Toe game:\n\n```javascript\n// tic-tac-toe.test.js\n\n// Import the functions to test\nconst { updateCell, changePlayer, checkWinner, restartGame } = require(\'./tic-tac-toe\');\n\ndescribe(\'Tic-Tac-Toe Game\', () => {\n    let options, currentPlayer, running;\n\n    beforeEach(() => {\n        options = Array(9).fill("");\n        currentPlayer = "tomato";\n        running = true;\n    });\n\n    test(\'sh

In [55]:
import json

output = messages.data[0].content[0].text.value
print(output)

To write a simple unit test for the Tic-Tac-Toe game, we'll use Jest, a popular JavaScript testing framework. We'll test the core functions such as `updateCell`, `changePlayer`, `checkWinner`, and `restartGame`.

Ensure that you have Jest installed in your project:
```bash
npm install --save-dev jest
```

Here's how you can create a unit test file for the Tic-Tac-Toe game:

```javascript
// tic-tac-toe.test.js

// Import the functions to test
const { updateCell, changePlayer, checkWinner, restartGame } = require('./tic-tac-toe');

describe('Tic-Tac-Toe Game', () => {
    let options, currentPlayer, running;

    beforeEach(() => {
        options = Array(9).fill("");
        currentPlayer = "tomato";
        running = true;
    });

    test('should update cell correctly', () => {
        const cell = { textContent: "" };
        updateCell(cell, 0, options, currentPlayer);
        expect(options[0]).toBe("tomato");
        expect(cell.textContent).toBe("tomato");
    });

    test('sh

Asking for running the unit test. 

In [56]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Run the unit test and show the results")

In [58]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [59]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_yc0Hge1JvkLAmmQRuanN1mI0', assistant_id='asst_VMZqGRm8uMlfSR0Qjg7oTA6e', attachments=[Attachment(file_id='file-3HmNja5XeXFEyexp77G0ZGSd', tools=[CodeInterpreterTool(type='code_interpreter')]), Attachment(file_id='file-Vj2fQEzx95KG5ozUur0I75kl', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=188, file_path=FilePath(file_id='file-Vj2fQEzx95KG5ozUur0I75kl'), start_index=156, text='sandbox:/mnt/data/tic-tac-toe.js', type='file_path'), FilePathAnnotation(end_index=263, file_path=FilePath(file_id='file-3HmNja5XeXFEyexp77G0ZGSd'), start_index=226, text='sandbox:/mnt/data/tic-tac-toe.test.js', type='file_path')], value="The refactored JavaScript file and test file have been saved. You can download them using the links below:\n\n- [Refactored JavaScript file (tic-tac-toe.js)](sandbox:/mnt/data/tic-tac-toe.js)\n- [Test file (tic-tac-toe.test.js)]

In [60]:
import json

output = messages.data[0].content[0].text.value
print(output)

The refactored JavaScript file and test file have been saved. You can download them using the links below:

- [Refactored JavaScript file (tic-tac-toe.js)](sandbox:/mnt/data/tic-tac-toe.js)
- [Test file (tic-tac-toe.test.js)](sandbox:/mnt/data/tic-tac-toe.test.js)

Currently, I don't have the capability to run JavaScript code or Jest tests directly. You can run the test locally by following these steps:

1. Ensure you have Node.js and npm installed.
2. Install Jest using the command:
    ```bash
    npm install --save-dev jest
    ```
3. Run Jest tests using the command:
    ```bash
    npx jest tic-tac-toe.test.js
    ```

This will execute the test cases and provide you with the test results in your terminal.
